In [1]:
%reload_ext autoreload
%autoreload 2
import os
import json
from pyeed.core import ProteinRecord

##### Here we get all the data

In [2]:
current_path = os.getcwd()

In [3]:
# read in the text file
file_path = 'TEM_Ids.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

# create a dic with the name and id
# A	TEM-P118	t		AAN05029 	AY130285 	12354869 	view			2be	ESBL	A
# we want to keep the name and the id, these are the second and the fourth columns
# we need to split the line by the tab character
data = {}
count = 0
for line in lines:
    line = line.strip()
    columns = line.split('\t')
    name = columns[1]
    id = columns[4].strip()
    if id == 'EFJ83682':
        continue
    data[name] = id
    count += 1


print('EFJ83682' in list(data.values()))

False


In [4]:
protein_ids_fetched = ProteinRecord.get_ids(list(data.values()))

# here we want to add the region for the ncbi blast id
# this is a kind of work arround to get the id later for the name mappping
for i in range(0, len(protein_ids_fetched)):
    protein_ids_fetched[i].add_to_regions(
        name="blastquery:" + list(data.keys())[i],
        start=0,
        end=len(protein_ids_fetched[i].sequence)
    )

print(protein_ids_fetched[0])

ProteinRecord
├── id = CBX53726.1
├── name = beta-lactamase TEM-40
├── organism
│   └── Organism
│       ├── id = 562
│       ├── taxonomy_id = 562
│       └── name = Escherichia coli
├── sequence = MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMISTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPAAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW
├── regions
│   ├── 0
│   │   └── Region
│   │       ├── id = PRK15442
│   │       ├── start = 2
│   │       └── end = 286
│   └── 1
│       └── Region
│           ├── id = 4366e534-1193-41c7-8f63-68a1a0dc3f9c
│           ├── name = blastquery:TEM-1
│           ├── start = 0
│           └── end = 286
├── coding_sequence
│   └── 0
│       └── Region
│           ├── id = FR717535.1
│           ├── start = 1
│           └── end = 861
└── ec_number = 3.5.2.6



In [5]:
current_path = os.path.dirname(os.getcwd())
print(len(protein_ids_fetched))

210


In [6]:
# we now search for each protein in the data dic a blast
# then we combine the results im data, we also have to remove double entries
data_proteins = []

"""
for protein in protein_ids_fetched:
    data_proteins = data_proteins + protein.ncbi_blast(n_hits=1000, e_value=0.1)
"""
"""
# no i want to parallelize this
import asyncio

async def fetch_protein(protein):
    return protein.ncbi_blast(n_hits=1000, e_value=0.1)

# create a job for each protein
jobs = [fetch_protein(protein) for protein in protein_ids_fetched]
# run all the jobs
data_proteins = await asyncio.gather(*jobs)
"""

"""
from concurrent.futures import ThreadPoolExecutor  # noqa: E402

def fetch_protein(protein):
    return protein.ncbi_blast(n_hits=150, e_value=0.1)


with ThreadPoolExecutor(max_workers=15) as executor:
    results = list(executor.map(fetch_protein, protein_ids_fetched))
"""


'\nfrom concurrent.futures import ThreadPoolExecutor  # noqa: E402\n\ndef fetch_protein(protein):\n    return protein.ncbi_blast(n_hits=150, e_value=0.1)\n\n\nwith ThreadPoolExecutor(max_workers=15) as executor:\n    results = list(executor.map(fetch_protein, protein_ids_fetched))\n'

In [7]:
"""
# add the protein_ids_fetched to results
for protein in protein_ids_fetched:
    results.append(protein)

data_proteins = results
# flatten the list
data_proteins = [item for sublist in data_proteins for item in sublist]
"""

data_proteins = protein_ids_fetched


In [8]:
# remove double entries
data_protein_unique = []
data_protein_unique_id = []
for protein in data_proteins:
    if type(protein) != ProteinRecord:
        print(protein)
        continue
    if protein.id not in data_protein_unique_id:
        data_protein_unique.append(protein)
        data_protein_unique_id.append(protein.id)

print(len(data_protein_unique))
print(data_protein_unique[0].id)

210
CBX53726.1


In [9]:
# read in the file from json
tem_dic = {}

with open(os.path.join(current_path, "TEM-lactamase", "TEM_Ids.json"), "r") as f:
    tem_dic = json.load(f)

In [10]:
data = {}

# create a dic with one field protein and one field 'dna'
for protein in data_protein_unique:
    try:
        data[tem_dic[protein.id.split('.')[0]]] = {
            'protein': protein,
            'dna': None
        }
    except KeyError:
        print(protein.id)

AFN69127.1


In [11]:
# now we pull the dna data
for key in data.keys():
    protein = data[key]['protein']
    dna = protein.get_dna()
    data[key]['dna'] = dna

In [12]:
# save the blast search results
output_folder_blast_search = os.path.join(current_path, "TEM-lactamase", "data", "data_fetch_list_ids_protein_dna_all")
os.makedirs(output_folder_blast_search, exist_ok=True)

def dumper(obj):
    try:
        return obj.json()
    except:
        return obj.__dict__

counter = 0
for key, value in data.items():
    with open(os.path.join(output_folder_blast_search, key + ".json"), "w") as f:
        json.dump(value, f, default=dumper)
        counter += 1